The following additional libraries are needed to run this
notebook. Note that running on Colab is experimental, please report a Github
issue if you have any problem.

In [ ]:
!pip install d2l==0.17.1


# Encontrando sinônimos e analogias
:label:`sec_synonyms`

Em :numref:`sec_word2vec_pretraining` treinamos um modelo de incorporação de palavras word2vec
em um conjunto de dados de pequena escala e procurou por sinônimos usando a similaridade de cosseno
de vetores de palavras. Na prática, vetores de palavras pré-treinados em um corpus de grande escala
muitas vezes pode ser aplicado a tarefas de processamento de linguagem natural downstream. Esta
seção irá demonstrar como usar esses vetores de palavras pré-treinados para encontrar
sinônimos e analogias. Continuaremos a aplicar vetores de palavras pré-treinados em
seções subsequentes.


In [1]:
import os
import torch
from torch import nn
from d2l import torch as d2l

## Usando vetores de palavras pré-treinados

Abaixo lista os embeddings GloVe pré-treinados de dimensões 50, 100 e 300,
que pode ser baixado do [site do GloVe](https://nlp.stanford.edu/projects/glove/).
Os fastText pré-treinados embarcados estão disponíveis em vários idiomas.
Aqui, consideramos uma versão em inglês ("wiki.en" 300-dimensional) que pode ser baixada do
[site fastText](https://fasttext.cc/).


In [2]:
#@save
d2l.DATA_HUB['glove.6b.50d'] = (d2l.DATA_URL + 'glove.6B.50d.zip',
                                '0b8703943ccdb6eb788e6f091b8946e82231bc4d')

#@save
d2l.DATA_HUB['glove.6b.100d'] = (d2l.DATA_URL + 'glove.6B.100d.zip',
                                 'cd43bfb07e44e6f27cbcc7bc9ae3d80284fdaf5a')

#@save
d2l.DATA_HUB['glove.42b.300d'] = (d2l.DATA_URL + 'glove.42B.300d.zip',
                                  'b5116e234e9eb9076672cfeabf5469f3eec904fa')

#@save
d2l.DATA_HUB['wiki.en'] = (d2l.DATA_URL + 'wiki.en.zip',
                           'c1816da3821ae9f43899be655002f6c723e91b88')

Definimos a seguinte classe `TokenEmbedding` para carregar os embeddings pré-treinados Glove e fastText acima.


In [3]:
#@save
class TokenEmbedding:
    """Token Embedding."""
    def __init__(self, embedding_name):
        self.idx_to_token, self.idx_to_vec = self._load_embedding(
            embedding_name)
        self.unknown_idx = 0
        self.token_to_idx = {token: idx for idx, token in
                             enumerate(self.idx_to_token)}

    def _load_embedding(self, embedding_name):
        idx_to_token, idx_to_vec = ['<unk>'], []
        data_dir = d2l.download_extract(embedding_name)
        # GloVe website: https://nlp.stanford.edu/projects/glove/
        # fastText website: https://fasttext.cc/
        with open(os.path.join(data_dir, 'vec.txt'), 'r') as f:
            for line in f:
                elems = line.rstrip().split(' ')
                token, elems = elems[0], [float(elem) for elem in elems[1:]]
                # Skip header information, such as the top row in fastText
                if len(elems) > 1:
                    idx_to_token.append(token)
                    idx_to_vec.append(elems)
        idx_to_vec = [[0] * len(idx_to_vec[0])] + idx_to_vec
        return idx_to_token, torch.tensor(idx_to_vec)

    def __getitem__(self, tokens):
        indices = [self.token_to_idx.get(token, self.unknown_idx)
                   for token in tokens]
        vecs = self.idx_to_vec[torch.tensor(indices)]
        return vecs

    def __len__(self):
        return len(self.idx_to_token)

Em seguida, usamos embeddings GloVe de 50 dimensões pré-treinados em um subconjunto da Wikipedia. A incorporação de palavras correspondente é baixada automaticamente na primeira vez que criamos uma instância de incorporação de palavras pré-treinada.


In [4]:
glove_6b50d = TokenEmbedding('glove.6b.50d')

Produza o tamanho do dicionário. O dicionário contém $400.000$ palavras e um token especial desconhecido.


In [5]:
len(glove_6b50d)

400001

Podemos usar uma palavra para obter seu índice no dicionário ou podemos obter a palavra de seu índice.


In [6]:
glove_6b50d.token_to_idx['beautiful'], glove_6b50d.idx_to_token[3367]

(3367, 'beautiful')

## Aplicação de vetores de palavras pré-treinados

Abaixo, demonstramos a aplicação de vetores de palavras pré-treinados, usando GloVe como exemplo.

### Encontrando sinônimos

Aqui, reimplementamos o algoritmo usado para pesquisar sinônimos por cosseno
similaridade introduzida em :numref:`sec_word2vec`

A fim de reutilizar a lógica para buscar os $k$ vizinhos mais próximos quando
buscando analogias, encapsulamos esta parte da lógica separadamente no `knn`
função ($k$-vizinhos mais próximos).


In [7]:
def knn(W, x, k):
    # The added 1e-9 is for numerical stability
    cos = torch.mv(W, x.reshape(-1,)) / (
        torch.sqrt(torch.sum(W * W, axis=1) + 1e-9) *
        torch.sqrt((x * x).sum()))
    _, topk = torch.topk(cos, k=k)
    return topk, [cos[int(i)] for i in topk]

Em seguida, buscamos sinônimos pré-treinando a instância do vetor de palavras `embed`.


In [8]:
def get_similar_tokens(query_token, k, embed):
    topk, cos = knn(embed.idx_to_vec, embed[[query_token]], k + 1)
    for i, c in zip(topk[1:], cos[1:]):  # Remove input words
        print(f'cosine sim={float(c):.3f}: {embed.idx_to_token[int(i)]}')

O dicionário de instância de vetor de palavras pré-treinadas `glove_6b50d` já criado contém 400.000 palavras e um token especial desconhecido. Excluindo palavras de entrada e palavras desconhecidas, procuramos as três palavras que têm o significado mais semelhante a "chip".


In [9]:
get_similar_tokens('chip', 3, glove_6b50d)

cosine sim=0.856: chips
cosine sim=0.749: intel
cosine sim=0.749: electronics


A seguir, procuramos os sinônimos de "baby" e "beautiful".


In [10]:
get_similar_tokens('baby', 3, glove_6b50d)

cosine sim=0.839: babies
cosine sim=0.800: boy
cosine sim=0.792: girl


In [11]:
get_similar_tokens('beautiful', 3, glove_6b50d)

cosine sim=0.921: lovely
cosine sim=0.893: gorgeous
cosine sim=0.830: wonderful


### Encontrando Analogias

Além de buscar sinônimos, também podemos usar o vetor de palavras pré-treinadas para buscar analogias entre palavras. Por exemplo, “man”:“woman”::“son”:“daughter” é um exemplo de analogia, “man” está para “woman” como “son” está para “daughter”. O problema de buscar analogias pode ser definido da seguinte forma: para quatro palavras na relação analógica $a : b :: c : d$, dadas as três primeiras palavras, $a$, $b$ e $c$, queremos encontre $d$. Suponha que a palavra vetor para a palavra $w$ seja $\text{vec}(w)$. Para resolver o problema de analogia, precisamos encontrar o vetor de palavras que é mais semelhante ao vetor de resultado de $\text{vec}(c)+\text{vec}(b)-\text{vec}(a)$.


In [12]:
def get_analogy(token_a, token_b, token_c, embed):
    vecs = embed[[token_a, token_b, token_c]]
    x = vecs[1] - vecs[0] + vecs[2]
    topk, cos = knn(embed.idx_to_vec, x, 1)
    return embed.idx_to_token[int(topk[0])]  # Remove unknown words

Verifique a analogia "male-female".


In [13]:
get_analogy('man', 'woman', 'son', glove_6b50d)

'daughter'

Analogia de “país-capital”: "beijing" é para "china" como "tokyo" é para quê? A resposta deve ser "japão".


In [14]:
get_analogy('beijing', 'china', 'tokyo', glove_6b50d)

'japan'

Analogia do "adjetivo-adjetivo superlativo": "ruim" está para o "pior", assim como "grande" está para o quê? A resposta deve ser "maior".


In [15]:
get_analogy('bad', 'worst', 'big', glove_6b50d)

'biggest'

Analogia do "verbo presente-verbo no pretérito": "do" é "did" assim como "go" é para quê? A resposta deve ser "went".


In [16]:
get_analogy('do', 'did', 'go', glove_6b50d)

'went'

## Sumário

* Vetores de palavras pré-treinados em um corpus de grande escala podem frequentemente ser aplicados a tarefas de processamento de linguagem natural downstream.
* Podemos usar vetores de palavras pré-treinados para buscar sinônimos e analogias.


## Exercícios

1. Teste os resultados do fastText usando `TokenEmbedding ('wiki.en')`.
1. Se o dicionário for extremamente grande, como podemos acelerar a localização de sinônimos e analogias?


[Discussão](https://discuss.d2l.ai/t/1336)


<!--stackedit_data:
eyJoaXN0b3J5IjpbLTEzNzYyMDg0OCw1MTk1NDA4MTFdfQ==
-->
